In [1]:
%load_ext autoreload
%autoreload 2

import sys
import torch
import pyro
sys.path.append('../')
sys.path.append('../Scripts')

from Scripts.new_sc_model import open_csv
from Scripts.new_sc_model import *

In [2]:
# datasets = { "bubble": ["Fig4"],
#              "four_waystrandexchange": ["Table5.2"],
#              "hairpin" : ["Fig4_0", "Fig4_1", "Fig6_0", "Fig6_1"], 
#              "hairpin1" : ["Fig3_T_0", "Fig3_T_1"],
#              "hairpin4" : ["Table1_0", "Table1_1"],
#              "helix" : ["Fig6_0", "Fig6_1"],
#              "helix1" : ["Fig6a"],
#              "three_waystranddisplacement" : ["Fig3b"], 
#              "three_waystranddisplacement1" : ["Fig6b"]
# }

datasets = { "hairpin" : ["Fig4_0"]}

In [3]:
def from_theta_to_rate(theta, datasets, kinetic_model="ARRHENIUS", stochastic_conditionning=False):
    
    # PATH = '/Users/aliseyfi/Documents/UBC/Probabilistic-Programming/Probabilistic-Programming/Project/'
    PATH = "C:/Users/jlovr/CS532-project/Probabilistic-Programming/Project/"
    predicted_log_10_rates, real_log_10_rates, errors = [], [], []
    for reaction_type in datasets:
            if reaction_type == "hairpin":
                for reaction_dataset in datasets[reaction_type]:
                    _zip = bool(int(reaction_dataset[-1]))
                    j = reaction_dataset[-3]
                    reaction_id = "/" + reaction_type + "/" + reaction_dataset
                    document_name = PATH + "/dataset" + reaction_id + ".csv"
                    file =  open_csv(document_name)
                    row = 1
                    while row < len(file) and file[row][0] != '' :
                        if row <= 10:
                            predicted_log_10_rate, real_log_10_rate, error = estimate_Bonnet(row, theta, _zip, file, reaction_id, str(row), "Bonnet"+j, kinetic_model, stochastic_conditionning)
                            predicted_log_10_rates.append(predicted_log_10_rate)
                            real_log_10_rates.append(real_log_10_rate)
                            errors.append(error)
                        row+=1
            else:
                pass

    return predicted_log_10_rates, real_log_10_rates, errors

In [ ]:
def model(data):

    theta_dim = 15

    # priors
    # theta_mean = [13.0580, 5, 17.0580, 5,  10.0580, 1, 1.0580, -2,  13.0580, 1, 5.0580, 0,  4.0580, -2,   0.0402 ]
    theta_mean = [13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ]
    theta = pyro.sample('theta', pyro.distributions.Normal(torch.tensor(theta_mean), torch.ones(theta_dim)))

    # likelihood
    ks, reals, errors = from_theta_to_rate(theta, data, stochastic_conditionning=True)
    for ind, k in enumerate(ks):
        # error = abs(k-reals[ind])
        error = errors[ind]
        # pyro.sample('obs_'+str(ind), pyro.distributions.Exponential(1), obs=torch.tensor(reals[ind]))
        # pyro.sample('obs_'+str(ind), pyro.distributions.Normal(k, 1), obs=torch.tensor(reals[ind]))
        # pyro.sample('obs_'+str(ind), pyro.distributions.Beta(1, 3), obs=torch.tensor(reals[ind]))
        pyro.sample('obs_'+str(ind), pyro.distributions.Normal(0, 1), obs=torch.tensor(error))

        # do a test to make sure abs(k-reals[ind]) under N(0,1) is the same as reals[ind] under Normal(k,1)

    return theta

In [ ]:
def model2(data):

    theta_dim = 15

    # priors
    # theta_mean = [13.0580, 5, 17.0580, 5,  10.0580, 1, 1.0580, -2,  13.0580, 1, 5.0580, 0,  4.0580, -2,   0.0402 ]
    theta_mean = [13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ]
    theta = pyro.sample('theta', pyro.distributions.Normal(torch.tensor(theta_mean), torch.ones(theta_dim)))

    # likelihood
    ks, reals, errors = from_theta_to_rate(theta, data, stochastic_conditionning=False)
    for ind, k in enumerate(ks):
        # error = abs(k-reals[ind])
        error = errors[ind]*errors[ind]
        # pyro.sample('obs_'+str(ind), pyro.distributions.Exponential(1), obs=torch.tensor(reals[ind]))
        # pyro.sample('obs_'+str(ind), pyro.distributions.Normal(k, 1), obs=torch.tensor(reals[ind]))
        # pyro.sample('obs_'+str(ind), pyro.distributions.Beta(1, 3), obs=torch.tensor(reals[ind]))
        pyro.sample('obs_'+str(ind), pyro.distributions.Normal(0, 1), obs=torch.tensor(error))

        # do a test to make sure abs(k-reals[ind]) under N(0,1) is the same as reals[ind] under Normal(k,1)

    return theta

In [ ]:
n_samples = 1

# Importance Sampling - original

In [ ]:
pyro.clear_param_store()
pyro.set_rng_seed(0)
# inference with importance sampling
importance = pyro.infer.Importance(model2, guide=None, num_samples=n_samples)

print("doing importance sampling...")
emp_marginal = pyro.infer.EmpiricalMarginal(importance.run(datasets))

posterior_mean = emp_marginal.mean
posterior_std_dev = emp_marginal.variance.sqrt()
ess = importance.get_ESS()


# report results
print(posterior_mean)
print(posterior_std_dev)
print("ess", ess)

print("done.")

# Importance Sampling - with stochastic conditionning

In [15]:
def ESS(W):
    summ = np.sum(W)
    W = [wi/summ for wi in W]
    return 1/np.sum([w**2 for w in W])

In [20]:
theta_dim = 15
n_samples = 100

samples,logWs = [],[]
theta_mean = [13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ]

for i in range(n_samples):
    theta = pyro.sample('theta', pyro.distributions.Normal(torch.tensor(theta_mean), torch.ones(theta_dim)))

    # likelihood
    ks, reals, errors = from_theta_to_rate(theta, datasets, stochastic_conditionning=True)
    loglik = 0

    for ind, k in enumerate(ks):
        # error = errors[ind]
        error = errors[ind]
        try:
            loglik += np.log(1 - error)
        except:
            loglik = -np.inf
    
    samples.append(theta)
    logWs.append(loglik)

print(samples)
print(logWs)
    

getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score
getting score


In [ ]:
def weighted_avg(X, weights):
    return (weights.dot(X)) / weights.sum()

for n in range(n_samples):
    samples[n] = np.array(samples[n], dtype=float)

W = np.exp(logWs)
means = weighted_avg(samples, W)
vars = weighted_avg((samples - means)**2, W)

print("mean", means)
print("variance", vars)


mean [11.6647625   3.59745693 12.73785877  3.43894148 13.29020977  2.80181003
 14.36070633  1.94243133 13.43860149  2.38264942 13.17202282  4.81843758
 11.63335896  3.9693923  -1.29722977]
variance [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.97215226e-31
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00]
ess 1.0
